In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box, shape
import rasterio
import os
import matplotlib.pyplot as plt
import pandas as pd
import rasterio.features

### Path setting

In [5]:
root_dir = Path(r"P:\bovenregionale-stresstest-hwn\Scripts\RA2CE_Model\data\Test_Run")
netwerkschakel_path = Path(r"P:\bovenregionale-stresstest-hwn\Data\Shapes netwerkschakels\Shapes netwerkschakels\HWN_netwerkindeling.shp")
study_area_path=Path(r'P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Gebiedsindeling obv waterschappen EXPORT 4dec24.gpkg')


static_path = root_dir.joinpath("static") #within your root directory there should be a 'static' folder
network_path = static_path.joinpath("network") #within the static folder there should be a 'network' folder
hazard_path = static_path.joinpath("hazard") #within the static folder there should be a 'hazard' folder
output_path = static_path.joinpath("output_graph")
assert root_dir.exists(), "root_dir not found."

### Loading data

In [6]:
study_area = gpd.read_file(study_area_path, driver='GPKG')
hazard_graph_read = gpd.read_file(output_path.joinpath("base_network_hazard.gpkg"), driver='GPKG')
gdf_schakels_segment = gpd.read_file(netwerkschakel_path, driver='ESRI Shapefile')

### Clipping on the specific study area

TO DO: check if it neatly clips the netwerkschakels so they 'extend' out of the area

In [7]:
# Dissolve geometries by NWSCODE to get the geometries of the netwerkschakels
gdf_schakels_all = gdf_schakels_segment.dissolve(by='NWSCODE', as_index=False)

# Select the study area geometry for 'Noord-Westelijke Delta'
study_area_zh = study_area[study_area['name'] == 'Noord-Westelijke Delta']
study_area_geom = study_area_zh.unary_union  # Combine into a single geometry

# Keep all geometries that intersect (touch or overlap) the study area
gdf_schakels = gdf_schakels_all[gdf_schakels_all.geometry.intersects(study_area_geom)]

# Reproject hazard graph to match CRS of study area
hazard_graph = hazard_graph_read.to_crs(study_area_zh.crs)

# Keep all hazard graph geometries that intersect the study area
hazard_graph = hazard_graph[hazard_graph.geometry.intersects(study_area_geom)]


### Setting flood thresholds to account for artefacts

TO DO if we get flood maps with different cell sizes: Make this flexible based on the flood map cell size and determine which cell size it is <br>
TO DO: Make a more 'scientific' approach to determining the correct threshold

Based on hazard graph

Determine suitable thresholds. The threshold notebook can help to test sensitivity to different threshold combinations

In [8]:
# --- Parameters ---
fraction_threshold = 0.3    # Minimum fraction flooded
depth_threshold = 0.2     # Minimum mean depth
zone_column = 'zone'      # Optional: column name for spatial aggregation

Determine for each segment if we should consider it 'flooded' based on a combination of threshold and whether the segment is 'isolated' in its floodedness

In [9]:
# --- Step 1: Load your data ---
gdf = hazard_graph.copy()

# --- Step 2: Filter flooded segments ---
gdf['is_flooded'] = (gdf['EV1_fr'] > fraction_threshold) & (gdf['EV1_me'] > depth_threshold)

# --- Step 3: Identify isolated flooded segments ---
def is_isolated(index, flooded_series):
    if not flooded_series.iloc[index]:
        return False
    neighbors = []
    if index > 0:
        neighbors.append(flooded_series.iloc[index - 1])
    if index < len(flooded_series) - 1:
        neighbors.append(flooded_series.iloc[index + 1])
    return not any(neighbors)

gdf['is_isolated'] = [is_isolated(i, gdf['is_flooded']) for i in range(len(gdf))]

# --- Step 4: Optional aggregation to zones ---
if zone_column in gdf.columns:
    zone_summary = gdf.groupby(zone_column).agg({
        'mean_depth': 'mean',
        'max_depth': 'max',
        'fraction_flooded': 'mean',
        'is_flooded': 'sum',
        'is_isolated': 'sum',
        # Example: keep first value of a non-numeric column
        'some_other_column': 'first'
    }).reset_index()

    # Optional: merge back to original GeoDataFrame
    gdf = gdf.merge(zone_summary, on=zone_column, suffixes=('', '_zone_summary'))

    print("Zone-level summary:")
    print(zone_summary)


### Join the segments to netwerkschakel
## TO UPDATE WITH NEW WORKFLOW: We should revise and update the base graphs with the nationaal wegenbestand attributes as these can be linked to the netwerkschakels!

### Also update the 'schakel_id' columns etc. 

In [10]:
import geopandas as gpd

# Ensure both layers use the same CRS
import geopandas as gpd

# --- Step 0: Reset intermediate variables if re-running ---
if 'gdf_segments_all' in locals():
    del gdf_segments_all, gdf_segments_crs, gdf_segments, gdf_joined

# --- Step 1: Prepare segment data ---
gdf_segments_all = gdf.copy()
gdf_segments_crs = gdf_segments_all.to_crs(gdf_schakels.crs)
gdf_segments = gdf_segments_crs.clip(mask=study_area_zh.geometry)

# --- Step 2: Ensure unique schakel ID ---
if 'schakel_id' not in gdf_schakels.columns:
    gdf_schakels = gdf_schakels.reset_index().rename(columns={'index': 'schakel_id'})

# --- Step 3: Spatial join (clean version) ---
# Drop previous join results if they exist
columns_to_remove = ['schakel_id']
gdf_segments = gdf_segments.drop(columns=[col for col in columns_to_remove if col in gdf_segments.columns])

# Perform spatial join
gdf_joined = gpd.sjoin(
    gdf_segments,
    gdf_schakels[['schakel_id', 'geometry']],
    how='left',
    predicate='intersects'
)

gdf_segments_all = gdf.copy()
gdf_segments_crs = gdf_segments_all.to_crs(gdf_schakels.crs)
gdf_segments = gdf_segments_crs.clip(mask=study_area_zh.geometry)

# Assign unique ID to schakels if not present
if 'schakel_id' not in gdf_schakels.columns:
    gdf_schakels = gdf_schakels.reset_index().rename(columns={'index': 'schakel_id'})

# Spatial join: assign each segment to a schakel
gdf_joined = gpd.sjoin(gdf_segments, gdf_schakels[['schakel_id', 'geometry']], how='left', predicate='intersects')


In [14]:
print(gdf_joined.groupby('schakel_id').size().sort_values(ascending=False).head())


schakel_id
103.0    155
113.0     98
41.0      94
101.0     83
194.0     79
dtype: int64


In [15]:
# --- Step 1: Aggregate flood metrics per schakel ---
schakel_summary = gdf_joined.groupby('schakel_id').agg(
    mean_depth=('EV1_me', 'mean'),
    max_depth=('EV1_ma', 'max'),
    fraction_flooded=('EV1_fr', 'mean'),
    num_flooded_segments=('is_flooded', lambda x: x.astype(bool).sum()),
    total_segments=('geometry', 'count')
).reset_index()


# --- Step 2: Determine if schakel is flooded ---
flooded_ratio_threshold = 0.1
schakel_summary['flooded_ratio'] = schakel_summary['num_flooded_segments'] / schakel_summary['total_segments']
schakel_summary['flooded_above_ratio_threshold'] = schakel_summary['flooded_ratio'] > flooded_ratio_threshold

# --- Step 3: Rank schakels by exposure ---
schakel_summary['exposure_score'] = (
    schakel_summary['flooded_ratio'].astype(float) * 0.5 +
    schakel_summary['mean_depth'] * 0.2 +
    schakel_summary['max_depth'] * 0.2
)

schakel_summary['rank'] = schakel_summary['exposure_score'].rank(ascending=True)

# --- Step 4: Clean merge with schakels GeoDataFrame ---
# Drop old columns if they exist to avoid duplication
columns_to_drop = [
    'mean_depth', 'max_depth', 'fraction_flooded',
    'num_flooded_segments', 'total_segments',
    'flooded_ratio', 'flooded_above_ratio_threshold',
    'exposure_score', 'rank'
]
gdf_schakels = gdf_schakels.drop(columns=[col for col in columns_to_drop if col in gdf_schakels.columns])

# Merge updated summary
gdf_schakels = gdf_schakels.merge(schakel_summary, on='schakel_id', how='left')

# --- Step 5: View top 5 most exposed schakels ---
print(gdf_schakels[['schakel_id', 'exposure_score', 'rank']].sort_values(by='rank').head())



    schakel_id  exposure_score  rank
30          84       -0.229865   1.0
24          40        0.037828   2.0
13         155        0.050255   3.0
27          98        0.060426   4.0
21         130        0.106679   5.0


In [18]:
schakel_summary.head()

,schakel_id,mean_depth,max_depth,fraction_flooded,num_flooded_segments,total_segments,flooded_ratio,flooded_above_ratio_threshold,exposure_score,rank
0,0.0,2.528274,11.027732,0.071853,2,25,0.080000,False,2.751201,21.0
1,30.0,NaN,NaN,0.000000,0,47,0.000000,False,NaN,NaN
2,37.0,1.746834,2.266313,0.043478,1,23,0.043478,False,0.824368,17.0
3,38.0,0.130834,0.435797,0.121804,3,74,0.040541,False,0.133596,7.0
4,39.0,0.079108,0.503665,0.017919,0,22,0.000000,False,0.116555,6.0


In [16]:
gdf_schakels.explore(column='num_flooded_segments', cmap='viridis')